In [1]:
//George Jen, Jen Tek LLC
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._

Intitializing Scala interpreter ...

Spark Web UI available at http://master.hadoop.lan:4040
SparkContext available as 'sc' (version = 3.0.0-preview, master = local[*], app id = local-1590642609971)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._


In [2]:
Logger.getLogger("org").setLevel(Level.ERROR)
val spark = SparkSession
    .builder
    .appName("genetic classifier")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", "file:///tmp")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@494cfca0


In [5]:
val ds = spark.read.format("csv").option("inferSchema", "true").option("header", "true")
  .option("quote", "\"")
    .load("file:///home/bigdata2/dataset/clinvar_conflicting.csv")
val df: DataFrame = ds.toDF()
df.show(3)

+-----+-------+---+---+------+-------+------+--------------------+------------+--------------------+---------+--------------------+----------+--------------------+--------------------+--------------------+------+---+-----+------+----------------+--------+-------+------------+--------------+--------------+----+------+-------------+------------+----------------+-----------+-------+--------+------+--------+--------------------+-----------------+----------+---------+------------+------------------+-------+----------+---------+--------+
|CHROM|    POS|REF|ALT|AF_ESP|AF_EXAC|AF_TGP|            CLNDISDB|CLNDISDBINCL|               CLNDN|CLNDNINCL|             CLNHGVS|CLNSIGINCL|               CLNVC|               CLNVI|                  MC|ORIGIN|SSR|CLASS|Allele|     Consequence|  IMPACT| SYMBOL|Feature_type|       Feature|       BIOTYPE|EXON|INTRON|cDNA_position|CDS_position|Protein_position|Amino_acids| Codons|DISTANCE|STRAND|BAM_EDIT|                SIFT|         PolyPhen|MOTIF_NAME|MOTIF

ds: org.apache.spark.sql.DataFrame = [CHROM: string, POS: int ... 44 more fields]
df: org.apache.spark.sql.DataFrame = [CHROM: string, POS: int ... 44 more fields]


In [6]:
df.groupBy("CLASS").count().show()

+-----+-----+
|CLASS|count|
+-----+-----+
|    1|16434|
|    0|48754|
+-----+-----+



In [7]:
df.printSchema

root
 |-- CHROM: string (nullable = true)
 |-- POS: integer (nullable = true)
 |-- REF: string (nullable = true)
 |-- ALT: string (nullable = true)
 |-- AF_ESP: double (nullable = true)
 |-- AF_EXAC: double (nullable = true)
 |-- AF_TGP: double (nullable = true)
 |-- CLNDISDB: string (nullable = true)
 |-- CLNDISDBINCL: string (nullable = true)
 |-- CLNDN: string (nullable = true)
 |-- CLNDNINCL: string (nullable = true)
 |-- CLNHGVS: string (nullable = true)
 |-- CLNSIGINCL: string (nullable = true)
 |-- CLNVC: string (nullable = true)
 |-- CLNVI: string (nullable = true)
 |-- MC: string (nullable = true)
 |-- ORIGIN: integer (nullable = true)
 |-- SSR: string (nullable = true)
 |-- CLASS: integer (nullable = true)
 |-- Allele: string (nullable = true)
 |-- Consequence: string (nullable = true)
 |-- IMPACT: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- Feature_type: string (nullable = true)
 |-- Feature: string (nullable = true)
 |-- BIOTYPE: string (nullable = t

In [10]:
df.columns.size

res7: Int = 46


In [11]:
//isAllDigits(df.select(df.columns(0)).collectAsList.get(0)(0).toString)

In [12]:
//def isNumeric(str:String): Boolean = str.matches("[-+]?\\d+(\\.\\d+)?")

In [13]:
import org.apache.spark.ml.feature.StringIndexer

import org.apache.spark.ml.feature.StringIndexer


In [14]:
df.dtypes

res10: Array[(String, String)] = Array((CHROM,StringType), (POS,IntegerType), (REF,StringType), (ALT,StringType), (AF_ESP,DoubleType), (AF_EXAC,DoubleType), (AF_TGP,DoubleType), (CLNDISDB,StringType), (CLNDISDBINCL,StringType), (CLNDN,StringType), (CLNDNINCL,StringType), (CLNHGVS,StringType), (CLNSIGINCL,StringType), (CLNVC,StringType), (CLNVI,StringType), (MC,StringType), (ORIGIN,IntegerType), (SSR,StringType), (CLASS,IntegerType), (Allele,StringType), (Consequence,StringType), (IMPACT,StringType), (SYMBOL,StringType), (Feature_type,StringType), (Feature,StringType), (BIOTYPE,StringType), (EXON,StringType), (INTRON,StringType), (cDNA_position,StringType), (CDS_position,StringType), (Protein_position,StringType), (Amino_acids,StringType), (Codons,StringType), (DISTANCE,IntegerType), (ST...


In [15]:
var df1=df.select("CLASS")
val featureDF=df.drop("CLASS")
val typeArray=featureDF.dtypes
for (i<-0 until featureDF.columns.size)
  {
    
//    println(typeArray(i)._2)
    if (typeArray(i)._2=="StringType")
      {
          var indexer = new StringIndexer().setInputCol(typeArray(i)._1).setOutputCol(typeArray(i)._1+"Index")
          var indexed = indexer.fit(featureDF.na.fill("ABCD").select(typeArray(i)._1)).transform(featureDF.na.fill("ABCD").select(typeArray(i)._1)) 
//          indexed.show(5)
          var temp=indexed.select(typeArray(i)._1+"index")
          df1=df1.withColumn("id", monotonically_increasing_id())
          .join(temp.withColumn("id", monotonically_increasing_id()), Seq("id"))
//          df1.show(5) 
      }
      else
      {
//          println(typeArray(i)._1) 
//          println(typeArray(i)._1.getClass)
          var temp=featureDF.na.fill(0).select(typeArray(i)._1)
          df1=df1.withColumn("id", monotonically_increasing_id()).join(temp.withColumn("id", monotonically_increasing_id()), Seq("id"))
//          df1.show(5)
      }
      
  }

df1: org.apache.spark.sql.DataFrame = [id: bigint, CLASS: int ... 45 more fields]
featureDF: org.apache.spark.sql.DataFrame = [CHROM: string, POS: int ... 43 more fields]
typeArray: Array[(String, String)] = Array((CHROM,StringType), (POS,IntegerType), (REF,StringType), (ALT,StringType), (AF_ESP,DoubleType), (AF_EXAC,DoubleType), (AF_TGP,DoubleType), (CLNDISDB,StringType), (CLNDISDBINCL,StringType), (CLNDN,StringType), (CLNDNINCL,StringType), (CLNHGVS,StringType), (CLNSIGINCL,StringType), (CLNVC,StringType), (CLNVI,StringType), (MC,StringType), (ORIGIN,IntegerType), (SSR,StringType), (Allele,StringType), (Consequence,StringType), (IMPACT,StringType), (SYMBOL,StringType), (Feature_type,StringType), (Feature,StringType), (BIOTYPE,StringType), (EXON,StringType), (INTRON,StringType), (cDNA_...


In [16]:
df1=df1.drop("id")

df1: org.apache.spark.sql.DataFrame = [CLASS: int, CHROMindex: double ... 44 more fields]


In [17]:
df1.printSchema

root
 |-- CLASS: integer (nullable = true)
 |-- CHROMindex: double (nullable = false)
 |-- POS: integer (nullable = false)
 |-- REFindex: double (nullable = false)
 |-- ALTindex: double (nullable = false)
 |-- AF_ESP: double (nullable = false)
 |-- AF_EXAC: double (nullable = false)
 |-- AF_TGP: double (nullable = false)
 |-- CLNDISDBindex: double (nullable = false)
 |-- CLNDISDBINCLindex: double (nullable = false)
 |-- CLNDNindex: double (nullable = false)
 |-- CLNDNINCLindex: double (nullable = false)
 |-- CLNHGVSindex: double (nullable = false)
 |-- CLNSIGINCLindex: double (nullable = false)
 |-- CLNVCindex: double (nullable = false)
 |-- CLNVIindex: double (nullable = false)
 |-- MCindex: double (nullable = false)
 |-- ORIGIN: integer (nullable = false)
 |-- SSRindex: double (nullable = false)
 |-- Alleleindex: double (nullable = false)
 |-- Consequenceindex: double (nullable = false)
 |-- IMPACTindex: double (nullable = false)
 |-- SYMBOLindex: double (nullable = false)
 |-- Featu

In [18]:
import org.apache.spark.sql.types._
val newDf = df1.select(df1.columns.map(c => col(c).cast(DoubleType)) : _*)




import org.apache.spark.sql.types._
newDf: org.apache.spark.sql.DataFrame = [CLASS: double, CHROMindex: double ... 44 more fields]


In [19]:
newDf.printSchema

root
 |-- CLASS: double (nullable = true)
 |-- CHROMindex: double (nullable = false)
 |-- POS: double (nullable = false)
 |-- REFindex: double (nullable = false)
 |-- ALTindex: double (nullable = false)
 |-- AF_ESP: double (nullable = false)
 |-- AF_EXAC: double (nullable = false)
 |-- AF_TGP: double (nullable = false)
 |-- CLNDISDBindex: double (nullable = false)
 |-- CLNDISDBINCLindex: double (nullable = false)
 |-- CLNDNindex: double (nullable = false)
 |-- CLNDNINCLindex: double (nullable = false)
 |-- CLNHGVSindex: double (nullable = false)
 |-- CLNSIGINCLindex: double (nullable = false)
 |-- CLNVCindex: double (nullable = false)
 |-- CLNVIindex: double (nullable = false)
 |-- MCindex: double (nullable = false)
 |-- ORIGIN: double (nullable = false)
 |-- SSRindex: double (nullable = false)
 |-- Alleleindex: double (nullable = false)
 |-- Consequenceindex: double (nullable = false)
 |-- IMPACTindex: double (nullable = false)
 |-- SYMBOLindex: double (nullable = false)
 |-- Feature_

In [20]:
newDf.show(2,false)

+-----+----------+---------+--------+--------+------+-------+------+-------------+-----------------+----------+--------------+------------+---------------+----------+----------+-------+------+--------+-----------+----------------+-----------+-----------+-----------------+------------+------------+---------+-----------+------------------+-----------------+---------------------+----------------+-----------+--------+------+-------------+---------+-------------+---------------+---------+-----------------+------------------+-------+----------+---------+--------+
|CLASS|CHROMindex|POS      |REFindex|ALTindex|AF_ESP|AF_EXAC|AF_TGP|CLNDISDBindex|CLNDISDBINCLindex|CLNDNindex|CLNDNINCLindex|CLNHGVSindex|CLNSIGINCLindex|CLNVCindex|CLNVIindex|MCindex|ORIGIN|SSRindex|Alleleindex|Consequenceindex|IMPACTindex|SYMBOLindex|Feature_typeindex|Featureindex|BIOTYPEindex|EXONindex|INTRONindex|cDNA_positionindex|CDS_positionindex|Protein_positionindex|Amino_acidsindex|Codonsindex|DISTANCE|STRAND|BAM_EDITinde

In [21]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors

val assembler = new VectorAssembler()
.setInputCols(Array("CHROMindex", "POS", "REFindex", "ALTindex", "AF_ESP", "AF_EXAC", "AF_TGP", "CLNDISDBindex"
        , "CLNDISDBINCLindex", "CLNDNindex", "CLNDNINCLindex", "CLNHGVSindex", "CLNSIGINCLindex", "CLNVCindex"
        , "CLNVIindex", "MCindex", "ORIGIN", "SSRindex", "Alleleindex", "Consequenceindex", "IMPACTindex"
        , "SYMBOLindex", "Feature_typeindex", "Featureindex", "BIOTYPEindex", "EXONindex", "INTRONindex"
        , "cDNA_positionindex", "CDS_positionindex", "Protein_positionindex", "Amino_acidsindex", "Codonsindex"
        , "DISTANCE", "STRAND", "BAM_EDITindex", "SIFTindex", "PolyPhenindex", "MOTIF_NAMEindex", "MOTIF_POS"
        , "HIGH_INF_POSindex", "MOTIF_SCORE_CHANGE", "LoFtool", "CADD_PHRED", "CADD_RAW", "BLOSUM62"))
.setOutputCol("features")
val output = assembler.transform(newDf)


import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_fb69990b52dd
output: org.apache.spark.sql.DataFrame = [CLASS: double, CHROMindex: double ... 45 more fields]


In [22]:
output.printSchema

root
 |-- CLASS: double (nullable = true)
 |-- CHROMindex: double (nullable = false)
 |-- POS: double (nullable = false)
 |-- REFindex: double (nullable = false)
 |-- ALTindex: double (nullable = false)
 |-- AF_ESP: double (nullable = false)
 |-- AF_EXAC: double (nullable = false)
 |-- AF_TGP: double (nullable = false)
 |-- CLNDISDBindex: double (nullable = false)
 |-- CLNDISDBINCLindex: double (nullable = false)
 |-- CLNDNindex: double (nullable = false)
 |-- CLNDNINCLindex: double (nullable = false)
 |-- CLNHGVSindex: double (nullable = false)
 |-- CLNSIGINCLindex: double (nullable = false)
 |-- CLNVCindex: double (nullable = false)
 |-- CLNVIindex: double (nullable = false)
 |-- MCindex: double (nullable = false)
 |-- ORIGIN: double (nullable = false)
 |-- SSRindex: double (nullable = false)
 |-- Alleleindex: double (nullable = false)
 |-- Consequenceindex: double (nullable = false)
 |-- IMPACTindex: double (nullable = false)
 |-- SYMBOLindex: double (nullable = false)
 |-- Feature_

In [23]:
output.select("features").show(2,false)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(45,[0,1,2,3,4,5,6,11,14,16,18,21,23,25,27,28,29,30,31,33,35,36,42,43,44],[3.0,1168180.0,1.0,3.0,0.0771,0.1002,0.1066,326.0,27451.0,1.0,3.0,1614.0,1728.0,15.0,618.0,471.0,304.0,56.0,129.0,1.0,2.0,1.0,1.053,-0.208682,2.0])|
|(45,[0,1,2,3,7,9,11,14,16,18,21,23,25,27,28,29,30,31,33,34,35,36,42,43,44],[3.0,1470752.0,1.0,1.0,7922.

In [26]:
val whole=output.select("features","CLASS")

whole: org.apache.spark.sql.DataFrame = [features: vector, CLASS: double]


In [27]:
whole.printSchema

root
 |-- features: vector (nullable = true)
 |-- CLASS: double (nullable = true)



In [28]:
/*
import org.apache.spark.ml.feature.StandardScaler
val scaler = new StandardScaler()
.setInputCol("features")
.setOutputCol("scaledFeatures")
.setWithStd(true)
.setWithMean(false)
// Compute summary statistics by fitting the StandardScaler.
val scalerModel = scaler.fit(whole)
// Normalize each feature to have unit standard deviation.
val scaledData = scalerModel.transform(whole)
scaledData.show(3)
*/

In [30]:
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.ml.linalg.Vectors
val scaler = new MinMaxScaler()
.setInputCol("features")
.setOutputCol("scaledFeatures")
// Compute summary statistics and generate MinMaxScalerModel
val scalerModel = scaler.fit(whole)
// rescale each feature to range [min, max].
val scaledData = scalerModel.transform(whole)
println(s"Features scaled to range: [${scaler.getMin}, ${scaler.getMax}]")
scaledData.select("features", "scaledFeatures").show(2,false)


Features scaled to range: [0.0, 1.0]
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                             

import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.ml.linalg.Vectors
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_e06d746dc2ed
scalerModel: org.apache.spark.ml.feature.MinMaxScalerModel = minMaxScal_e06d746dc2ed
scaledData: org.apache.spark.sql.DataFrame = [features: vector, CLASS: double ... 1 more field]


In [31]:
scaledData.printSchema

root
 |-- features: vector (nullable = true)
 |-- CLASS: double (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



In [32]:
val wholeScaledData=scaledData.select("scaledFeatures","CLASS")

wholeScaledData: org.apache.spark.sql.DataFrame = [scaledFeatures: vector, CLASS: double]


In [33]:
// Prepare training and test data.
val Array(training, test) = wholeScaledData.randomSplit(Array(0.7, 0.3), seed = 12345)


training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [scaledFeatures: vector, CLASS: double]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [scaledFeatures: vector, CLASS: double]


In [35]:
training.show(3,false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaledFeatures                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [36]:
test.show(3,false)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaledFeatures                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [37]:
//val layers = Array[Int](4, 5, 4, 3)
val layers = Array[Int](45, 45, 45, 2)

layers: Array[Int] = Array(45, 45, 45, 2)


In [38]:
import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val trainer = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setBlockSize(128)
  .setSeed(1234L)
  .setMaxIter(100)
  .setFeaturesCol("scaledFeatures")
  .setLabelCol("CLASS")

import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
trainer: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_e67cb70b6901


In [39]:
val model=trainer.fit(training.select("scaledFeatures","CLASS"))

model: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_e67cb70b6901


In [40]:
// compute accuracy on the test set
val result = model.transform(test)


result: org.apache.spark.sql.DataFrame = [scaledFeatures: vector, CLASS: double ... 3 more fields]


In [41]:
result.printSchema

root
 |-- scaledFeatures: vector (nullable = true)
 |-- CLASS: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [42]:
val predictionAndLabels = result.select("prediction", "CLASS")


predictionAndLabels: org.apache.spark.sql.DataFrame = [prediction: double, CLASS: double]


In [43]:
val evaluator = new MulticlassClassificationEvaluator()
  .setMetricName("accuracy").setLabelCol("CLASS")



evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1cb0298ba121


In [44]:
println(s"Test set accuracy = ${evaluator.evaluate(predictionAndLabels)}")

Test set accuracy = 0.7482470955524848


In [45]:
import org.apache.spark.ml.{Pipeline, PipelineStage}
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel}
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.ml.{Pipeline, PipelineStage}
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel}
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.sql.{DataFrame, SparkSession}


In [46]:
import scala.collection.mutable
val maxIter=100
val lr = new LogisticRegression()
      .setFeaturesCol("scaledFeatures")
      .setLabelCol("CLASS")
      .setMaxIter(maxIter)

import scala.collection.mutable
maxIter: Int = 100
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_00807e29abd2


In [47]:
val model_lr = lr.fit(training)

model_lr: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_00807e29abd2, numClasses = 2, numFeatures = 45


In [48]:
val prediction_lr = model_lr.transform(test)

prediction_lr: org.apache.spark.sql.DataFrame = [scaledFeatures: vector, CLASS: double ... 3 more fields]


In [49]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


In [ ]:
val my_eval_aoc = new BinaryClassificationEvaluator()
  .setLabelCol("CLASS")
  .setRawPredictionCol("prediction").setMetricName("areaUnderROC")

my_eval_aoc.evaluate(prediction_lr)

In [ ]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
var my_mc_f1 = new MulticlassClassificationEvaluator().setPredictionCol("prediction").setLabelCol("CLASS").setMetricName("f1")
my_mc_f1.evaluate(prediction_lr)

In [50]:
val my_mc_accu = new MulticlassClassificationEvaluator().setPredictionCol("prediction").setLabelCol("CLASS").setMetricName("accuracy")
my_mc_accu.evaluate(prediction_lr)

my_mc_accu: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_74ebac4de5a0
res29: Double = 0.7463534469522494
